In [1]:
# run in docker:
# sudo docker run -p 8890:8888 -v /home/tehdog/data/dev/2019/pic2kcal-cv-praktikum:/tf/notebooks --runtime=nvidia -it --rm tensorflow/tensorflow:1.12.0-gpu-py3 jupyter notebook --allow-root --notebook-dir=/tf/notebooks
# crashes in TF1.4!
# does not work with Arch Linux TF installation!

In [2]:
%cd ..

/home/veheusser/Code_Projects/cvhci_praktikum/pic2kcal


In [3]:
!pip install tensorflow-hub
!pip install sentencepiece
!pip install tf-sentencepiece
!pip install tqdm

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.
DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7.


In [4]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  en_de_embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling/en-de/1")
  embedded_text = en_de_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)

ModuleNotFoundError: No module named 'tf_sentencepiece'

In [ ]:
from pathlib import Path
import json
import re
import random
import heapq
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm
from math import ceil

data_dir = Path("data")

In [ ]:
# jq '.[]' fddb_data_v3.json > fddb_data_v3.jsonl
# jq 'select(([.Bilder[]|select(.title != "Noch kein Foto vorhanden.")]|length) > 0)' fddb_data_v3.jsonl | jq -s > fddb_data_v3_withimg.json
with open(str(data_dir / "fddb_data_v3_withimg.json"), encoding='utf-8') as f:
    fddb = json.load(f)
    # todo: make unique here
    _out_names = [e["name"] for e in fddb]

# jq '.ingredients[]|select(.ingredient)|.ingredient' processed_data.jsonl | jq -s unique > ingredients.json
with open(str(data_dir / "recipes/ingredients.json"), encoding='utf-8') as f:
    _in_names = json.load(f)


In [ ]:
def normalize_ingredient(ing: str):
    ing = re.sub(r"\(([^)])\)", "\g<1>", ing)  # remove stuff in parens
    ing = re.sub(r"\([^)]+\)", "", ing)  # remove stuff in parens
    ing = re.sub(r"(\d+,)?\d+ k?g\b", "", ing)  # remove xyz gram
    ing = re.sub(r",.*", "", ing)
    ing = re.sub(r"\bzum .*", "", ing)
    ing = re.sub(r"\boder\b.*", "", ing)
    ing = ing.strip()
    return ing

from extract_ingredients.util import normalize_out_ingredient
    

In [ ]:
in_names = list({normalize_ingredient(ing) for ing in _in_names})
print(len(_in_names))
in_names.sort()
len(in_names)

In [ ]:
print(len(_out_names))
out_names = list({ning for ing in _out_names for ning in normalize_out_ingredient(ing)})
out_names.sort()
len(out_names)

In [ ]:
#def Matcher():
#    def __init__(self, data_left, data_right, preproc_left, preproc_right):
#        pre_left = list(map(preproc_left, data_left))
#        self.left_vecs = list(zip(data_left, pre_left, get_sentence_vectors(pre_left)))
#        
#        pre_right = list(map(preproc_right, data_right))
#        self.right_vecs = list(zip(data_right, pre_right, get_sentence_vectors(pre_right)))

In [ ]:
def make_chunks(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

In [ ]:
def get_sentence_vectors(texts):
    bs = 10000
    ccount = len(texts)//bs
    chunks = make_chunks(texts, bs)
    if ccount >= 3:
        chunks = tqdm(chunks, total=ccount)
    for chunk in chunks:
        yield from session.run(embedded_text, feed_dict={text_input: chunk})

In [ ]:
def get_match(search: np.array, out_vecs, limit=10):
    it = ((v[0], np.dot(v[1], search)) for v in out_vecs)
    res_list = heapq.nlargest(limit, it, key=itemgetter(1))
    return res_list

In [ ]:
out_vecs = list(zip(out_names, get_sentence_vectors(out_names)))

In [ ]:
out_vecs[0][1].shape

In [ ]:
in_names[0], out_names[0]

In [ ]:
in_vecs = list(zip(in_names, get_sentence_vectors(in_names)))

In [ ]:
for ingredient, vec in random.sample(in_vecs, 100):
    res_list = get_match(vec, out_vecs)
    print("{} -> {}".format(ingredient, res_list[0]))

In [ ]:
z = [inv for inv in in_vecs if inv[0] == 'Kartoffeln'][0]
get_match(z[1], out_vecs)

In [ ]:
z = list(get_sentence_vectors(["Schalotten"]))[0]
get_match(z, out_vecs, limit=30)

In [ ]:
z = list(get_sentence_vectors([normalize_ingredient("Salz")]))[0]
get_match(z, out_vecs, limit=30)

In [ ]:
# i thought multiprocessing is impossible in jupyter notebook?
# better not question it
from multiprocessing import Pool

def jsonable(e):
    return [(a,float(b)) for a, b in e]

def get_match_w(t):
    ing, vec = t
    return ing, get_match(vec, out_vecs)

all_ings = {}
with Pool(8) as pool:
    for ing, match in pool.imap(get_match_w, tqdm(in_vecs), chunksize=100):
        all_ings[ing] = jsonable(match)

In [ ]:

all_ings_orig = {ing: all_ings[normalize_ingredient(ing)] for ing in _in_names}

In [ ]:
with open("data/recipe-ingredient-to-fooddb.json", "w") as f:
    json.dump(all_ings_orig, f)